## 재무정보(전체 재무제표) - 단일 회사(상장기업) - 반기 보고서
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/

### 1. 대상 회사/연도/보고서 설정 및 재무제표 정보 가져오기

In [11]:
import requests
import pandas as pd

fiscal_year = '2023'
listed_corp_codes = \
['01208885',    # 경동도시가스
 '00787376',    # 대성에너지
 '00128555',    # 삼천리
 '00130684']    # 서울도시가스
report_codes = '11012'   # 반기보고서

In [12]:
# 재무제표 가져오기
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
df_fi_all = pd.DataFrame()

for corp_code in listed_corp_codes:
    for report_code in report_codes:
        params = {
            'corp_code': corp_code,
            'bsns_year': fiscal_year,
            'reprt_code': report_code,
            'fs_div': 'OFS',            # OFS:재무제표
            'crtfc_key': api_key,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:             # URL GET 200 : 정상
            print("URL GET Error")
        if response.json()['status'] == '000':      # 데이터 유무 '000' : 정상
            # 데이터프레임 형태로 변환
            df_fi = pd.json_normalize(response.json()['list'])
            if not df_fi.empty:
                df_fi_all = pd.concat([df_fi_all, df_fi], ignore_index=True)

In [13]:
if not df_fi_all.empty:
    print(df_fi_all.size)
else:
    print("가져온 데이터가 없습니다.")

55860


### 2. 데이터 전처리 및 엑셀 저장

In [14]:
if not df_fi_all.empty:
    # 불필요한 컬럼 삭제
    # df_fi_all.drop(labels=['account_id'], axis=1, inplace=True)     # 필수 항목. 삭제하면 안 됨. 코드 수정(2023.07.24)

    # 문자열 컬럼에 있는 null값을 ''로 변환
    df_fi_all['thstrm_nm'].fillna('', inplace=True)
    df_fi_all['frmtrm_nm'].fillna('', inplace=True)
    df_fi_all['frmtrm_q_nm'].fillna('', inplace=True)

    # 금액 컬럼에 있는 null값을 ''로 변환
    df_fi_all['thstrm_amount'].fillna('', inplace=True)
    df_fi_all['frmtrm_amount'].fillna('', inplace=True)
    df_fi_all['frmtrm_q_amount'].fillna('', inplace=True)
    df_fi_all['thstrm_add_amount'].fillna('', inplace=True)
    df_fi_all['frmtrm_add_amount'].fillna('', inplace=True)
    
    # 금액 컬럼에 있는 ''을 '0'로 변환(저장 시에 숫자 형식으로 변환을 위해서임)
    df_fi_all = df_fi_all.replace({'thstrm_amount' : '',
                                  'frmtrm_amount' : '',
                                  'frmtrm_q_amount' : '',
                                  'thstrm_add_amount' : '',
                                  'frmtrm_add_amount' : ''}, '0')
    # 타입 변환
    df_fi_all = df_fi_all.astype({'thstrm_amount' : 'float',
                                  'frmtrm_amount' : 'float',
                                  'thstrm_add_amount' : 'float',
                                  'thstrm_add_amount' : 'float',
                                  'frmtrm_add_amount' : 'float'})
    
    df_fi_all = df_fi_all.astype({'ord' : 'int'})

In [15]:
df_fi_all

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
0,20230814002096,1,2023,01208885,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 7 기 반기말,3.456650e+11,제 6 기말,6.870267e+11,1,KRW,0.0,,0,0.0
1,20230814002096,1,2023,01208885,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 7 기 반기말,5.832869e+10,제 6 기말,8.927760e+10,2,KRW,0.0,,0,0.0
2,20230814002096,1,2023,01208885,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 7 기 반기말,1.142000e+11,제 6 기말,1.092000e+11,3,KRW,0.0,,0,0.0
3,20230814002096,1,2023,01208885,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,당기손익-공정가치금융자산,-,제 7 기 반기말,3.303904e+10,제 6 기말,7.011508e+10,4,KRW,0.0,,0,0.0
4,20230814002096,1,2023,01208885,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 7 기 반기말,1.319898e+11,제 6 기말,4.076840e+11,5,KRW,0.0,,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,20230814001424,2,2023,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|기타자본항목,제 41 기 반기,-4.544518e+10,,0.000000e+00,9,KRW,0.0,제 40 기 반기,-45445180487,0.0
2936,20230814001424,2,2023,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|기타포괄손익누계액 [member],제 41 기 반기,-4.353054e+09,,0.000000e+00,9,KRW,0.0,제 40 기 반기,-4353351581,0.0
2937,20230814001424,2,2023,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|이익잉여금 [member],제 41 기 반기,1.047058e+12,,0.000000e+00,9,KRW,0.0,제 40 기 반기,1033332968918,0.0
2938,20230814001424,2,2023,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|자본금 [member],제 41 기 반기,3.500000e+10,,0.000000e+00,9,KRW,0.0,제 40 기 반기,35000000000,0.0


In [16]:
df_fi_all.to_excel('D:\PythonProject\data-gatherer\dart\data\co_fi_all_multiple.xlsx')